## CycleGAN for Image translation with Residual Blocks and TensorFlow 2

In [ ]:
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.layers import Input, Conv2D, Conv2DTranspose, Activation, LeakyReLU, Concatenate
from keras.initializers import RandomNormal

In [ ]:
# install keras.contrib with the InstanceNormalization layer
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
# import the new instance normalization layer
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

In [ ]:
# create the discriminator for the GAN
def discriminator(input_shape):
    # kernel initializer for our layers
    init = RandomNormal(stddev = 0.02)
    # image input
    inputs = Input(shape = input_shape)
    # first conv layer with 32 filters
    x = Conv2D(16, 5, 2, padding='same', kernel_initializer=init, name='conv_0')(inputs)
    x = LeakyReLU(alpha=0.2)(x)
    # conv layers with instance normalization
    x = Conv2D(32, 5, 2, padding='same', kernel_initializer=init, name='conv_1')(x)
    x = InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(64, 5, 2, padding='same', kernel_initializer=init, name='conv_2')(x)
    x = InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(128, 5, 2, padding='same', kernel_initializer=init, name='conv_3')(x)
    x = InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(256, 5, padding='same', kernel_initializer=init, name='conv_5')(x)
    x = InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    # output layer
    outputs = Conv2D(1, 5, padding='same', kernel_initializer=init, name='output')(x)
    # define the discriminator
    model = Model(inputs, outputs)
    # compile the model
    model.compile(loss='mse', optimizer=Adam(lr=0.0001, beta_1=0.5))
    return model

In [ ]:
# define image shape constant
IMG_SHAPE = (256, 256, 3)
# declare the model
model = discriminator(IMG_SHAPE)
# model summary
model.summary()

In [ ]:
# create residual blocks for the generator
def res_block(filters, inputs):
    # kernel weights initializer
    init = RandomNormal(stddev=0.02)
    x = Conv2D(filters, 3, padding='same', kernel_initializer=init)(inputs)
    x = InstanceNormalization(axis=-1)(x)
    x = Activation('selu')(x)
    x = Conv2D(filters, 3, padding='same', kernel_initializer=init)(x)
    x = InstanceNormalization(axis=-1)(x)
    # concatenate second conv layer with the inputs
    x = Concatenate()([x, inputs])
    return x

In [ ]:
# generator function
def generator(img_shape = (256, 256, 3), n_blocks = 8):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    inputs = Input(shape = (256, 256, 3))
    x = Conv2D(16, 5, padding='same', kernel_initializer=init)(inputs)
    x = InstanceNormalization(axis=-1)(x)
    x = Activation('selu')(x)
    
    x = Conv2D(32, 3, 2, padding='same', kernel_initializer=init)(x)
    x = InstanceNormalization(axis=-1)(x)
    x = Activation('selu')(x)
    
    x = Conv2D(64, 3, 2, padding='same', kernel_initializer=init)(x)
    x = InstanceNormalization(axis=-1)(x)
    x = Activation('selu')(x)
    
    # add residual blocks to our generator
    for _ in range(n_blocks):
        x = res_block(128, x)
    
    # transpose convolutions
    x = Conv2DTranspose(32, 3, strides = 2, padding='same', kernel_initializer=init)(x)
    x = InstanceNormalization(axis=-1)(x)
    x = Activation('selu')(x)
    
    x = Conv2DTranspose(64, 3, 2, padding='same', kernel_initializer=init)(x)
    x = InstanceNormalization(axis=-1)(x)
    x = Activation('selu')(x)
    
    # output layer
    x = Conv2D(3, 7, padding='same', kernel_initializer=init)(x)
    x = InstanceNormalization(axis=-1)(x)
    outputs = Activation('tanh')(x)
    
    # create the model
    model = Model(inputs, outputs)
    return model

In [ ]:
model = generator()
model.summary()

In [ ]:
%cd ..
!ls 

In [ ]:
# define the two generators and two discriminators for CycleGAN
generator_BtoA = generator(IMG_SHAPE)
generator_AtoB = generator(IMG_SHAPE)
discriminator_A = discriminator(IMG_SHAPE)
discriminator_B = discriminator(IMG_SHAPE)

In [ ]:
# define composite model
def composite_model(g1, d1, g2, image_shape):
    # mark the first generator as trainable
	g1.trainable = True
	# freeze discriminator
	d1.trainable = False
	# freeze second generator
	g2.trainable = False
	input_gen = Input(shape=image_shape)
	gen1_out = g1(input_gen)
	output_d = d1(gen1_out)
	input_id = Input(shape=image_shape)
	output_id = g1(input_id)
	output_f = g2(gen1_out)
	gen2_out = g2(input_id)
	output_b = g1(gen2_out)
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	optimizer = Adam(lr=0.0001, beta_1=0.5)
	# compile model with L1 loss
	model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=optimizer)
	return model

In [ ]:
from PIL import Image
import os
import numpy as np

monet_path = 'input/gan-getting-started/monet_jpg/'
monet_array = []

# monet images to npz
for _, file in enumerate(os.listdir(monet_path)): 
    image = Image.open(monet_path + file)
    single_array = np.array(image)
    monet_array.append(single_array)
# save the images
np.savez('monet_compressed.npz', monet_array)

In [ ]:
import numpy as np
x1 = np.load('monet_compressed.npz')

In [ ]:
# photos to npz
photo_path = 'input/gan-getting-started/photo_jpg/'
photo_array_1 = []
c = 0
# photo images to npz (limit the length to prevent memory error)
for _, file in enumerate(os.listdir(photo_path)): 
    c+=1
    image = Image.open(photo_path + file)
    single_array_1 = np.array(image)
    photo_array_1.append(single_array_1)
    if c == 3000:
        break
# save the images
np.savez('photos1_compressed.npz', photo_array_1)

In [ ]:
# load the photos
x2 = np.load('photos1_compressed.npz')

In [ ]:
# scale the data to -1 and 1

X1 = (x1['arr_0'] - 127.5) / 127.5

In [ ]:
X2 = (x2['arr_0'] - 127.5) / 127.5

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# translate an image without training
plt.axis(False)
translation = generator_BtoA(np.reshape(X2[1], (1, 256, 256, 3)))
print('TRANSLATED IMAGE WITHOUT TRAINING')
plt.imshow(np.reshape(translation, (256, 256, 3)))

In [ ]:
print('ORIGINAL IMAGE')
plt.axis(False)
plt.imshow(X2[1])

In [ ]:
from numpy.random import randint
def generate_real_samples(dataset, n_samples, patch_shape):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = np.ones((n_samples, patch_shape, patch_shape, 1))
	return X, y

In [ ]:
# generate fake images (batch size)
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate the image
	X = g_model(dataset)
	# create 'fake' class labels (0)
	y = np.zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [ ]:
def summarize_performance(step, g_model, trainX, name, n_samples=5):
	X_in, _ = generate_real_samples(trainX, n_samples, 0)
	# generate target images
	X_out, _ = generate_fake_samples(g_model, X_in, 0)
	# rescale to 0 - 1
	X_in = (X_in + 1) / 2.0
	X_out = (X_out + 1) / 2.0
	for i in range(n_samples):
		plt.subplot(2, n_samples, 1 + i)
		plt.axis('off')
		plt.imshow(X_in[i])
	# plot target image
	for i in range(n_samples):
		plt.subplot(2, n_samples, 1 + n_samples + i)
		plt.axis('off')
		plt.imshow(X_out[i])
	# save plot to file
	filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
	plt.savefig(filename1)
	plt.close()

In [ ]:
# create a pool of fake images to trace the training
from random import random
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return np.asarray(selected)

In [ ]:
# train the cycleGAN models for 50 epochs with a batch size of 1
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
    # train for 10 epochs
	n_epochs, n_batch, = 10, 1
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
		# generate fake images
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update images
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance every so often
		if (i+1) % (bat_per_epo * 1) == 0:
			# plot A->B translation
			summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
			# plot B->A translation
			summarize_performance(i, g_model_BtoA, trainB, 'BtoA')

In [ ]:
# create the composite model for A to B
composite_AtoB = composite_model(generator_AtoB, discriminator_B,generator_BtoA, IMG_SHAPE)
composite_BtoA = composite_model(generator_BtoA, discriminator_A, generator_AtoB, IMG_SHAPE)

In [ ]:
# finally, train the model
train(discriminator_A, discriminator_B, generator_AtoB,generator_BtoA,  composite_AtoB,
     composite_BtoA, [X1, X2])

In [ ]:
# save the model
%cd working
generator_BtoA.save('monet_translator.h5')

In [ ]:
translation = generator_BtoA(np.reshape(X2[600], (1, 256, 256, 3)))
print('TRANSLATED IMAGE AFTER TRAINING')
plt.imshow(np.reshape(translation, (256, 256, 3)))

In [ ]:
print('ORIGINAL BASE IMAGE')
plt.imshow(X2[600])